<a href="https://colab.research.google.com/github/KOreshOK9922/flow-deviation/blob/main/%D0%90%D0%9D%D0%90%D0%9B%D0%98%D0%97_%D0%92%D0%9B%D0%98%D0%AF%D0%9D%D0%98%D0%AF_%D0%A1%D0%A2%D0%A0%D0%A3%D0%9A%D0%A2%D0%A3%D0%A0%D0%AB_%D0%9A%D0%90%D0%9D%D0%90%D0%9B%D0%9E%D0%92_%D0%98_%D0%9A%D0%9E%D0%A0%D0%A0%D0%95%D0%9B%D0%AF%D0%A6%D0%98%D0%98_%D0%9D%D0%90_%D0%9E%D0%A2%D0%9A%D0%9B%D0%9E%D0%9D%D0%95%D0%9D%D0%98%D0%95_21_01_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import pearsonr

# Константы
Q_VALUES = [50.0, 100.0, 150.0, 200.0, 250.0, 300.0, 350.0, 400.0, 450.0, 500.0]
Q_NV_VALUES = [40, 50, 60]

# Типы грузов
CARGO_TYPES = [
    {"name": "Каменный уголь", "a_p": 1.224, "a_o": 1.066, "b": 0.660},
    {"name": "Нефть", "a_p": 1.260, "a_o": 1.160, "b": 0.658},
    {"name": "Руда", "a_p": 1.293, "a_o": 1.188, "b": 0.657},
    {"name": "Черные металлы", "a_p": 1.249, "a_o": 1.090, "b": 0.652},
    {"name": "Лесные", "a_p": 1.232, "a_o": 1.076, "b": 0.676},
    {"name": "Минеральные и строительные материалы", "a_p": 1.393, "a_o": 1.225, "b": 0.653},
    {"name": "Химические и минеральные удобрения", "a_p": 1.289, "a_o": 1.128, "b": 0.642},
    {"name": "Хлебные", "a_p": 1.420, "a_o": 1.241, "b": 0.662},
    {"name": "Прочие", "a_p": 1.320, "a_o": 1.139, "b": 0.701}
]

# Структуры каналов
CHANNEL_STRUCTURES = {
    "1-канал": {"channels": 1, "distribution": [1.0]},
    "2-канала (10/90)": {"channels": 2, "distribution": [0.1, 0.9]},
    "2-канала (30/70)": {"channels": 2, "distribution": [0.3, 0.7]},
    "2-канала (50/50)": {"channels": 2, "distribution": [0.5, 0.5]},
    "3-канала (10/20/70)": {"channels": 3, "distribution": [0.1, 0.2, 0.7]},
    "3-канала (20/30/50)": {"channels": 3, "distribution": [0.2, 0.3, 0.5]},
    "3-канала (33.3)": {"channels": 3, "distribution": [1 / 3, 1 / 3, 1 / 3]}
}

# Генерация данных
def generate_dataset():
    data = []
    days_in_year = 365
    for q in Q_VALUES:
        for structure_name, structure in CHANNEL_STRUCTURES.items():
            distribution = structure["distribution"]
            num_channels = structure["channels"]
            avg_daily_flow = q
            channel_flows = [avg_daily_flow * dist for dist in distribution]
            for day in range(1, days_in_year + 1):
                for channel_idx in range(num_channels):
                    intended_mean = channel_flows[channel_idx]
                    value = np.random.normal(loc=intended_mean, scale=intended_mean * 0.05)
                    data.append({
                        "q": q,
                        "Структура каналов": structure_name,
                        "День": day,
                        "Канал": channel_idx + 1,
                        "Значение": value
                    })
    return pd.DataFrame(data)

# Функция для построения графиков частот
def plot_frequencies(df, pdf_pages):
    for structure_name, structure in CHANNEL_STRUCTURES.items():
        num_channels = structure["channels"]
        if num_channels == 1:
            fig, ax = plt.subplots(figsize=(10, 5))
            channel_data = df[(df['Структура каналов'] == structure_name) & (df['Канал'] == 1)]['Значение']
            sns.histplot(channel_data, kde=True, ax=ax)
            ax.set_title(f"Частотный график для структуры каналов: {structure_name}, Канал 1")
            ax.set_xlabel("Значение")
            ax.set_ylabel("Частота")
            plt.tight_layout()
            pdf_pages.savefig(fig)
            plt.close(fig)
        else:
            fig, axes = plt.subplots(1, num_channels, figsize=(15, 5))
            fig.suptitle(f"Частотные графики для структуры каналов: {structure_name}")
            for channel_idx in range(num_channels):
                channel_data = df[(df['Структура каналов'] == structure_name) & (df['Канал'] == channel_idx + 1)]['Значение']
                sns.histplot(channel_data, kde=True, ax=axes[channel_idx])
                axes[channel_idx].set_title(f"Канал {channel_idx + 1}")
                axes[channel_idx].set_xlabel("Значение")
                axes[channel_idx].set_ylabel("Частота")
            plt.tight_layout()
            pdf_pages.savefig(fig)
            plt.close(fig)

# Сохранение данных
def save_dataset(df, filename="generated_data.csv"):
    df.to_csv(filename, index=False)
    print(f"Данные сохранены в файл {filename}")

# Загрузка данных
def load_dataset(filename="generated_data.csv"):
    return pd.read_csv(filename)

# Расчет корреляции
def calculate_correlation(x, y):
    return pearsonr(x, y)[0]

# Расчет сигмы для одного канала
def calculate_single_channel_sigma(cargo, m):
    return {
        'sigma_p': cargo["a_p"] * (m ** cargo["b"]),
        'sigma_o': cargo["a_o"] * (m ** cargo["b"])
    }

# Расчет сигмы для двух каналов без корреляции
def calculate_two_channel_sigma_without_corr(cargo, m, dist):
    sigma_1_p = cargo["a_p"] * ((m * dist[0]) ** cargo["b"])
    sigma_2_p = cargo["a_p"] * ((m * dist[1]) ** cargo["b"])
    sigma_p = np.sqrt((sigma_1_p ** 2) + (sigma_2_p ** 2))

    sigma_1_o = cargo["a_o"] * ((m * dist[0]) ** cargo["b"])
    sigma_2_o = cargo["a_o"] * ((m * dist[1]) ** cargo["b"])
    sigma_o = np.sqrt((sigma_1_o ** 2) + (sigma_2_o ** 2))

    return {'sigma_p': sigma_p, 'sigma_o': sigma_o}

# Расчет сигмы для трех каналов без корреляции
def calculate_three_channel_sigma_without_corr(cargo, m, dist):
    sigma_1_p = cargo["a_p"] * ((m * dist[0]) ** cargo["b"])
    sigma_2_p = cargo["a_p"] * ((m * dist[1]) ** cargo["b"])
    sigma_3_p = cargo["a_p"] * ((m * dist[2]) ** cargo["b"])
    sigma_p = np.sqrt((sigma_1_p ** 2) + (sigma_2_p ** 2) + (sigma_3_p ** 2))

    sigma_1_o = cargo["a_o"] * ((m * dist[0]) ** cargo["b"])
    sigma_2_o = cargo["a_o"] * ((m * dist[1]) ** cargo["b"])
    sigma_3_o = cargo["a_o"] * ((m * dist[2]) ** cargo["b"])
    sigma_o = np.sqrt((sigma_1_o ** 2) + (sigma_2_o ** 2) + (sigma_3_o ** 2))

    return {'sigma_p': sigma_p, 'sigma_o': sigma_o}

# Расчет сигмы для двух каналов с корреляцией
def calculate_two_channel_sigma_with_corr(cargo, m, dist, corr):
    sigma_1_p = cargo["a_p"] * ((m * dist[0]) ** cargo["b"])
    sigma_2_p = cargo["a_p"] * ((m * dist[1]) ** cargo["b"])
    sigma_p = np.sqrt((sigma_1_p ** 2) + (sigma_2_p ** 2) + (2 * corr * sigma_1_p * sigma_2_p))

    sigma_1_o = cargo["a_o"] * ((m * dist[0]) ** cargo["b"])
    sigma_2_o = cargo["a_o"] * ((m * dist[1]) ** cargo["b"])
    sigma_o = np.sqrt((sigma_1_o ** 2) + (sigma_2_o ** 2) + (2 * corr * sigma_1_o * sigma_2_o))

    return {'sigma_p': sigma_p, 'sigma_o': sigma_o}

# Расчет сигмы для трех каналов с корреляцией
def calculate_three_channel_sigma_with_corr(cargo, m, dist, corr):
    sigma_1_p = cargo["a_p"] * ((m * dist[0]) ** cargo["b"])
    sigma_2_p = cargo["a_p"] * ((m * dist[1]) ** cargo["b"])
    sigma_3_p = cargo["a_p"] * ((m * dist[2]) ** cargo["b"])
    sigma_p = np.sqrt((sigma_1_p ** 2) + (sigma_2_p ** 2) + (sigma_3_p ** 2) +
                      2 * corr[0][1] * sigma_1_p * sigma_2_p +
                      2 * corr[0][2] * sigma_1_p * sigma_3_p +
                      2 * corr[1][2] * sigma_2_p * sigma_3_p)

    sigma_1_o = cargo["a_o"] * ((m * dist[0]) ** cargo["b"])
    sigma_2_o = cargo["a_o"] * ((m * dist[1]) ** cargo["b"])
    sigma_3_o = cargo["a_o"] * ((m * dist[2]) ** cargo["b"])
    sigma_o = np.sqrt((sigma_1_o ** 2) + (sigma_2_o ** 2) + (sigma_3_o ** 2) +
                      2 * corr[0][1] * sigma_1_o * sigma_2_o +
                      2 * corr[0][2] * sigma_1_o * sigma_3_o +
                      2 * corr[1][2] * sigma_2_o * sigma_3_o)

    return {'sigma_p': sigma_p, 'sigma_o': sigma_o}

# Расчет энтропии
def calculate_H(sigma):
    return np.log(sigma * np.sqrt(2 * np.pi * np.e))

# Основная функция для расчетов
def calculate_cargo_deviations_combined():
    results = []
    df = load_dataset()

    for cargo in CARGO_TYPES:
        for q in Q_VALUES:
            for q_nv in Q_NV_VALUES:
                m = q / q_nv
                for structure_name, structure in CHANNEL_STRUCTURES.items():
                    distribution = structure.get("distribution", None)
                    num_channels = structure["channels"]

                    if num_channels == 1:
                        sigmas = calculate_single_channel_sigma(cargo, m)
                        H_p = calculate_H(sigmas['sigma_p'])
                        H_o = calculate_H(sigmas['sigma_o'])
                        results.append({
                            "Груз": cargo["name"],
                            "Суточный грузопоток q (т/сутки)": q,
                            "Технорма грузоподъемности q_nv (т)": q_nv,
                            "m (q/q_nv)": round(m, 2),
                            "Структура каналов": structure_name,
                            "Отклонение для прибытия σ_p (без корреляции)": round(sigmas['sigma_p'], 6),
                            "Отклонение для отправления σ_o (без корреляции)": round(sigmas['sigma_o'], 6),
                            "H для прибытия (без корреляции)": round(H_p, 6),
                            "H для отправления (без корреляции)": round(H_o, 6),
                            "Отклонение для прибытия σ_p (с корреляцией)": None,
                            "Отклонение для отправления σ_o (с корреляцией)": None,
                            "H для прибытия (с корреляцией)": None,
                            "H для отправления (с корреляцией)": None
                        })
                    elif num_channels == 2:
                        sigmas_without_corr = calculate_two_channel_sigma_without_corr(cargo, m, distribution)
                        H_p_without_corr = calculate_H(sigmas_without_corr['sigma_p'])
                        H_o_without_corr = calculate_H(sigmas_without_corr['sigma_o'])

                        df_filtered = df[(df['q'] == q) & (df['Структура каналов'] == structure_name)]
                        correlations = np.zeros((num_channels, num_channels))
                        for i in range(num_channels):
                            for j in range(num_channels):
                                if i == j:
                                    correlations[i][j] = 1.0
                                else:
                                    channel_i_data = df_filtered[df_filtered['Канал'] == i + 1]['Значение']
                                    channel_j_data = df_filtered[df_filtered['Канал'] == j + 1]['Значение']
                                    correlations[i][j] = calculate_correlation(channel_i_data, channel_j_data)

                        sigmas_with_corr = calculate_two_channel_sigma_with_corr(cargo, m, distribution, correlations[0][1])
                        H_p_with_corr = calculate_H(sigmas_with_corr['sigma_p'])
                        H_o_with_corr = calculate_H(sigmas_with_corr['sigma_o'])

                        results.append({
                            "Груз": cargo["name"],
                            "Суточный грузопоток q (т/сутки)": q,
                            "Технорма грузоподъемности q_nv (т)": q_nv,
                            "m (q/q_nv)": round(m, 2),
                            "Структура каналов": structure_name,
                            "Отклонение для прибытия σ_p (без корреляции)": round(sigmas_without_corr['sigma_p'], 6),
                            "Отклонение для отправления σ_o (без корреляции)": round(sigmas_without_corr['sigma_o'], 6),
                            "H для прибытия (без корреляции)": round(H_p_without_corr, 6),
                            "H для отправления (без корреляции)": round(H_o_without_corr, 6),
                            "Отклонение для прибытия σ_p (с корреляцией)": round(sigmas_with_corr['sigma_p'], 6),
                            "Отклонение для отправления σ_o (с корреляцией)": round(sigmas_with_corr['sigma_o'], 6),
                            "H для прибытия (с корреляцией)": round(H_p_with_corr, 6),
                            "H для отправления (с корреляцией)": round(H_o_with_corr, 6)
                        })
                    elif num_channels == 3:
                        sigmas_without_corr = calculate_three_channel_sigma_without_corr(cargo, m, distribution)
                        H_p_without_corr = calculate_H(sigmas_without_corr['sigma_p'])
                        H_o_without_corr = calculate_H(sigmas_without_corr['sigma_o'])

                        df_filtered = df[(df['q'] == q) & (df['Структура каналов'] == structure_name)]
                        correlations = np.zeros((num_channels, num_channels))
                        for i in range(num_channels):
                            for j in range(num_channels):
                                if i == j:
                                    correlations[i][j] = 1.0
                                else:
                                    channel_i_data = df_filtered[df_filtered['Канал'] == i + 1]['Значение']
                                    channel_j_data = df_filtered[df_filtered['Канал'] == j + 1]['Значение']
                                    correlations[i][j] = calculate_correlation(channel_i_data, channel_j_data)

                        sigmas_with_corr = calculate_three_channel_sigma_with_corr(cargo, m, distribution, correlations)
                        H_p_with_corr = calculate_H(sigmas_with_corr['sigma_p'])
                        H_o_with_corr = calculate_H(sigmas_with_corr['sigma_o'])

                        results.append({
                            "Груз": cargo["name"],
                            "Суточный грузопоток q (т/сутки)": q,
                            "Технорма грузоподъемности q_nv (т)": q_nv,
                            "m (q/q_nv)": round(m, 2),
                            "Структура каналов": structure_name,
                            "Отклонение для прибытия σ_p (без корреляции)": round(sigmas_without_corr['sigma_p'], 6),
                            "Отклонение для отправления σ_o (без корреляции)": round(sigmas_without_corr['sigma_o'], 6),
                            "H для прибытия (без корреляции)": round(H_p_without_corr, 6),
                            "H для отправления (без корреляции)": round(H_o_without_corr, 6),
                            "Отклонение для прибытия σ_p (с корреляцией)": round(sigmas_with_corr['sigma_p'], 6),
                            "Отклонение для отправления σ_o (с корреляцией)": round(sigmas_with_corr['sigma_o'], 6),
                            "H для прибытия (с корреляцией)": round(H_p_with_corr, 6),
                            "H для отправления (с корреляцией)": round(H_o_with_corr, 6)
                        })
    return pd.DataFrame(results)

def calculate_and_save_changes(results_df, excel_path="cargo_deviations_changes.xlsx"):
    changes_df = []

    for cargo in CARGO_TYPES:
        cargo_name = cargo["name"]
        for q_nv in Q_NV_VALUES:
            for q in Q_VALUES:
                base_structure = "1-канал"
                base_data = results_df[
                    (results_df['Груз'] == cargo_name) &
                    (results_df['Технорма грузоподъемности q_nv (т)'] == q_nv) &
                    (results_df['Суточный грузопоток q (т/сутки)'] == q) &
                    (results_df['Структура каналов'] == base_structure)
                ]

                for structure_name in CHANNEL_STRUCTURES.keys():
                    if structure_name == base_structure:
                        continue

                    structure_data = results_df[
                        (results_df['Груз'] == cargo_name) &
                        (results_df['Технорма грузоподъемности q_nv (т)'] == q_nv) &
                        (results_df['Суточный грузопоток q (т/сутки)'] == q) &
                        (results_df['Структура каналов'] == structure_name)
                    ]

                    if not base_data.empty and not structure_data.empty:
                        changes = {
                            "Груз": cargo_name,
                            "Суточный грузопоток q (т/сутки)": q,
                            "Технорма грузоподъемности q_nv (т)": q_nv,
                            "Структура каналов": structure_name,
                            "Изменение σ_p (без корреляции) (%)": round(((structure_data['Отклонение для прибытия σ_p (без корреляции)'].values[0] - base_data['Отклонение для прибытия σ_p (без корреляции)'].values[0]) / base_data['Отклонение для прибытия σ_p (без корреляции)'].values[0]) * 100, 2),
                            "Изменение σ_o (без корреляции) (%)": round(((structure_data['Отклонение для отправления σ_o (без корреляции)'].values[0] - base_data['Отклонение для отправления σ_o (без корреляции)'].values[0]) / base_data['Отклонение для отправления σ_o (без корреляции)'].values[0]) * 100, 2),
                            "Изменение H для прибытия (без корреляции) (%)": round(((structure_data['H для прибытия (без корреляции)'].values[0] - base_data['H для прибытия (без корреляции)'].values[0]) / base_data['H для прибытия (без корреляции)'].values[0]) * 100, 2),
                            "Изменение H для отправления (без корреляции) (%)": round(((structure_data['H для отправления (без корреляции)'].values[0] - base_data['H для отправления (без корреляции)'].values[0]) / base_data['H для отправления (без корреляции)'].values[0]) * 100, 2),
                            "Изменение σ_p (с корреляцией) относительно без корреляции (%)": round(((structure_data['Отклонение для прибытия σ_p (с корреляцией)'].values[0] - structure_data['Отклонение для прибытия σ_p (без корреляции)'].values[0]) / structure_data['Отклонение для прибытия σ_p (без корреляции)'].values[0]) * 100, 2),
                            "Изменение σ_o (с корреляцией) относительно без корреляции (%)": round(((structure_data['Отклонение для отправления σ_o (с корреляцией)'].values[0] - structure_data['Отклонение для отправления σ_o (без корреляции)'].values[0]) / structure_data['Отклонение для отправления σ_o (без корреляции)'].values[0]) * 100, 2),
                            "Изменение H для прибытия (с корреляцией) относительно без корреляции (%)": round(((structure_data['H для прибытия (с корреляцией)'].values[0] - structure_data['H для прибытия (без корреляции)'].values[0]) / structure_data['H для прибытия (без корреляции)'].values[0]) * 100, 2),
                            "Изменение H для отправления (с корреляцией) относительно без корреляции (%)": round(((structure_data['H для отправления (с корреляцией)'].values[0] - structure_data['H для отправления (без корреляции)'].values[0]) / structure_data['H для отправления (без корреляции)'].values[0]) * 100, 2)
                        }
                        changes_df.append(changes)

    changes_df = pd.DataFrame(changes_df)
    changes_df.to_excel(excel_path, index=False, sheet_name='Изменения')
    print(f"Изменения сохранены в файл {excel_path}")
    return changes_df  # Возвращаем changes_df


# Функция для построения графиков
def plot_sigma_and_H(results_df, pdf_path="cargo_analysis.pdf"):
    with PdfPages(pdf_path) as pdf:
        for cargo in CARGO_TYPES:
            cargo_name = cargo["name"]
            for q_nv in Q_NV_VALUES:
                fig, axes = plt.subplots(3, 2, figsize=(20, 20))
                fig.suptitle(f"Анализ для груза: {cargo_name}, q_nv = {q_nv}", fontsize=16)

                cargo_data = results_df[
                    (results_df['Груз'] == cargo_name) &
                    (results_df['Технорма грузоподъемности q_nv (т)'] == q_nv)
                ]

                colors = plt.cm.tab10.colors

                # График 1: Отклонение для прибытия σ_p (без корреляции и с корреляцией) от q
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    axes[0, 0].plot(
                        structure_data['Суточный грузопоток q (т/сутки)'],
                        structure_data['Отклонение для прибытия σ_p (без корреляции)'],
                        label=f"{structure_name} (без корреляции)",
                        linestyle='-', marker='o', color=colors[i]
                    )
                    axes[0, 0].plot(
                        structure_data['Суточный грузопоток q (т/сутки)'],
                        structure_data['Отклонение для прибытия σ_p (с корреляцией)'],
                        label=f"{structure_name} (с корреляции)",
                        linestyle='--', marker='x', color=colors[i]
                    )
                axes[0, 0].set_title("Отклонение для прибытия σ_p от q")
                axes[0, 0].set_xlabel("Суточный грузопоток q (т/сутки)")
                axes[0, 0].set_ylabel("Отклонение для прибытия σ_p")
                axes[0, 0].legend()
                axes[0, 0].grid(True)

                # График 2: Отклонение для отправления σ_o (без корреляции и с корреляцией) от q
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    axes[0, 1].plot(
                        structure_data['Суточный грузопоток q (т/сутки)'],
                        structure_data['Отклонение для отправления σ_o (без корреляции)'],
                        label=f"{structure_name} (без корреляции)",
                        linestyle='-', marker='o', color=colors[i]
                    )
                    axes[0, 1].plot(
                        structure_data['Суточный грузопоток q (т/сутки)'],
                        structure_data['Отклонение для отправления σ_o (с корреляцией)'],
                        label=f"{structure_name} (с корреляции)",
                        linestyle='--', marker='x', color=colors[i]
                    )
                axes[0, 1].set_title("Отклонение для отправления σ_o от q")
                axes[0, 1].set_xlabel("Суточный грузопоток q (т/сутки)")
                axes[0, 1].set_ylabel("Отклонение для отправления σ_o")
                axes[0, 1].legend()
                axes[0, 1].grid(True)

                # График 3: H для прибытия (без корреляции) от σ_p (без корреляции)
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    axes[1, 0].plot(
                        structure_data['Отклонение для прибытия σ_p (без корреляции)'],
                        structure_data['H для прибытия (без корреляции)'],
                        label=f"{structure_name} (без корреляции)",
                        linestyle='-', marker='o', color=colors[i]
                    )
                axes[1, 0].set_title("H для прибытия (без корреляции) от σ_p (без корреляции)")
                axes[1, 0].set_xlabel("Отклонение для прибытия σ_p (без корреляции)")
                axes[1, 0].set_ylabel("H для прибытия (без корреляции)")
                axes[1, 0].legend()
                axes[1, 0].grid(True)

                # График 4: H для прибытия (с корреляцией) от σ_p (с корреляцией)
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    axes[1, 1].plot(
                        structure_data['Отклонение для прибытия σ_p (с корреляцией)'],
                        structure_data['H для прибытия (с корреляцией)'],
                        label=f"{structure_name} (с корреляции)",
                        linestyle='--', marker='x', color=colors[i]
                    )
                axes[1, 1].set_title("H для прибытия (с корреляцией) от σ_p (с корреляцией)")
                axes[1, 1].set_xlabel("Отклонение для прибытия σ_p (с корреляцией)")
                axes[1, 1].set_ylabel("H для прибытия (с корреляцией)")
                axes[1, 1].legend()
                axes[1, 1].grid(True)

                # График 5: H для отправления (без корреляции) от σ_o (без корреляции)
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    axes[2, 0].plot(
                        structure_data['Отклонение для отправления σ_o (без корреляции)'],
                        structure_data['H для отправления (без корреляции)'],
                        label=f"{structure_name} (без корреляции)",
                        linestyle='-', marker='o', color=colors[i]
                    )
                axes[2, 0].set_title("H для отправления (без корреляции) от σ_o (без корреляции)")
                axes[2, 0].set_xlabel("Отклонение для отправления σ_o (без корреляции)")
                axes[2, 0].set_ylabel("H для отправления (без корреляции)")
                axes[2, 0].legend()
                axes[2, 0].grid(True)

                # График 6: H для отправления (с корреляцией) от σ_o (с корреляцией)
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    axes[2, 1].plot(
                        structure_data['Отклонение для отправления σ_o (с корреляцией)'],
                        structure_data['H для отправления (с корреляцией)'],
                        label=f"{structure_name} (с корреляции)",
                        linestyle='--', marker='x', color=colors[i]
                    )
                axes[2, 1].set_title("H для отправления (с корреляцией) от σ_o (с корреляцией)")
                axes[2, 1].set_xlabel("Отклонение для отправления σ_o (с корреляцией)")
                axes[2, 1].set_ylabel("H для отправления (с корреляцией)")
                axes[2, 1].legend()
                axes[2, 1].grid(True)

                plt.tight_layout()
                pdf.savefig(fig)
                plt.close()

    print(f"Графики сохранены в файл {pdf_path}")


def plot_changes(changes_df, pdf_path="changes_analysis.pdf"):
    with PdfPages(pdf_path) as pdf:
        for cargo in CARGO_TYPES:
            cargo_name = cargo["name"]
            fig, axes = plt.subplots(3, 4, figsize=(25, 18))  # Уменьшен размер фигуры

            # Добавляем общее название фигуры
            fig.suptitle(f"Изменение параметров относительно структуры каналов для груза: {cargo_name}",
                         fontsize=16, y=1.02)

            colors = plt.cm.tab10.colors

            for idx, q_nv in enumerate(Q_NV_VALUES):
                cargo_data = changes_df[
                    (changes_df['Груз'] == cargo_name) &
                    (changes_df['Технорма грузоподъемности q_nv (т)'] == q_nv)
                ]

                # График 1: Изменение σ_p (без корреляции) от структуры каналов
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    if structure_name == "1-канал":
                        continue
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    if not structure_data.empty and 'Изменение σ_p (без корреляции) (%)' in structure_data.columns:
                        axes[idx, 0].bar(
                            structure_name.replace("канала", ""),  # Убираем слово "канала"
                            structure_data['Изменение σ_p (без корреляции) (%)'].values[0],
                            label=f"{structure_name}",
                            color=colors[i]
                        )
                axes[idx, 0].set_title(f"Изменение σ_p (без корреляции) для q_nv = {q_nv}")
                axes[idx, 0].set_xlabel("Структура")  # Убрано слово "канала"
                axes[idx, 0].set_ylabel("Значение в % для σ_p")  # Изменена подпись оси ординат
                axes[idx, 0].legend()
                axes[idx, 0].grid(True)

                # График 2: Изменение σ_o (без корреляции) от структуры каналов
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    if structure_name == "1-канал":
                        continue
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    if not structure_data.empty and 'Изменение σ_o (без корреляции) (%)' in structure_data.columns:
                        axes[idx, 1].bar(
                            structure_name.replace("канала", ""),  # Убираем слово "канала"
                            structure_data['Изменение σ_o (без корреляции) (%)'].values[0],
                            label=f"{structure_name}",
                            color=colors[i]
                        )
                axes[idx, 1].set_title(f"Изменение σ_o (без корреляции) для q_nv = {q_nv}")
                axes[idx, 1].set_xlabel("Структура")  # Убрано слово "канала"
                axes[idx, 1].set_ylabel("Значение в % для σ_o")  # Изменена подпись оси ординат
                axes[idx, 1].legend()
                axes[idx, 1].grid(True)

                # График 3: Изменение H для прибытия (без корреляции) от структуры каналов
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    if structure_name == "1-канал":
                        continue
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    if not structure_data.empty and 'Изменение H для прибытия (без корреляции) (%)' in structure_data.columns:
                        axes[idx, 2].bar(
                            structure_name.replace("канала", ""),  # Убираем слово "канала"
                            structure_data['Изменение H для прибытия (без корреляции) (%)'].values[0],
                            label=f"{structure_name}",
                            color=colors[i]
                        )
                axes[idx, 2].set_title(f"Изменение H для прибытия (без корреляции) для q_nv = {q_nv}")
                axes[idx, 2].set_xlabel("Структура")  # Убрано слово "канала"
                axes[idx, 2].set_ylabel("Значение в % для H прибытия")  # Изменена подпись оси ординат
                axes[idx, 2].legend()
                axes[idx, 2].grid(True)

                # График 4: Изменение H для отправления (без корреляции) от структуры каналов
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    if structure_name == "1-канал":
                        continue
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    if not structure_data.empty and 'Изменение H для отправления (без корреляции) (%)' in structure_data.columns:
                        axes[idx, 3].bar(
                            structure_name.replace("канала", ""),  # Убираем слово "канала"
                            structure_data['Изменение H для отправления (без корреляции) (%)'].values[0],
                            label=f"{structure_name}",
                            color=colors[i]
                        )
                axes[idx, 3].set_title(f"Изменение H для отправления (без корреляции) для q_nv = {q_nv}")
                axes[idx, 3].set_xlabel("Структура")  # Убрано слово "канала"
                axes[idx, 3].set_ylabel("Значение в % для H отправления")  # Изменена подпись оси ординат
                axes[idx, 3].legend()
                axes[idx, 3].grid(True)

            plt.tight_layout(pad=3.0)  # Увеличено пространство между графиками
            pdf.savefig(fig)
            plt.close()

    print(f"Графики изменений сохранены в файл {pdf_path}")


def plot_changes_with_corr_vs_without_corr(changes_df, pdf_path="changes_with_corr_vs_without_corr.pdf"):
    with PdfPages(pdf_path) as pdf:
        for cargo in CARGO_TYPES:
            cargo_name = cargo["name"]
            fig, axes = plt.subplots(3, 4, figsize=(25, 18))  # Уменьшен размер фигуры
            fig.suptitle(f"Изменение параметров с учетом корреляции для груза {cargo_name}", fontsize=20, y=0.98)

            colors = plt.cm.tab10.colors

            for idx, q_nv in enumerate(Q_NV_VALUES):
                cargo_data = changes_df[
                    (changes_df['Груз'] == cargo_name) &
                    (changes_df['Технорма грузоподъемности q_nv (т)'] == q_nv)
                ]

                # График 1: Изменение σ_p (с корреляцией) относительно без корреляции от структуры каналов
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    if structure_name == "1-канал":
                        continue
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    if not structure_data.empty and 'Изменение σ_p (с корреляцией) относительно без корреляции (%)' in structure_data.columns:
                        axes[idx, 0].bar(
                            structure_name.replace("канала", ""),  # Удаляем слово "канал"
                            structure_data['Изменение σ_p (с корреляцией) относительно без корреляции (%)'].values[0],
                            label=f"{structure_name}",
                            color=colors[i]
                        )
                axes[idx, 0].set_title(f"Изменение σ_p (с корреляцией) для q_nv = {q_nv}", fontsize=12, pad=10)
                axes[idx, 0].set_xlabel("Структура", fontsize=10)
                axes[idx, 0].set_ylabel("Значение в % для σ_p", fontsize=10)
                axes[idx, 0].legend(fontsize=8)
                axes[idx, 0].grid(True)

                # График 2: Изменение σ_o (с корреляцией) относительно без корреляции от структуры каналов
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    if structure_name == "1-канал":
                        continue
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    if not structure_data.empty and 'Изменение σ_o (с корреляцией) относительно без корреляции (%)' in structure_data.columns:
                        axes[idx, 1].bar(
                            structure_name.replace("канала", ""),  # Удаляем слово "канал"
                            structure_data['Изменение σ_o (с корреляцией) относительно без корреляции (%)'].values[0],
                            label=f"{structure_name}",
                            color=colors[i]
                        )
                axes[idx, 1].set_title(f"Изменение σ_o (с корреляцией) для q_nv = {q_nv}", fontsize=12, pad=10)
                axes[idx, 1].set_xlabel("Структура", fontsize=10)
                axes[idx, 1].set_ylabel("Значение в % для σ_o", fontsize=10)
                axes[idx, 1].legend(fontsize=8)
                axes[idx, 1].grid(True)

                # График 3: Изменение H для прибытия (с корреляцией) относительно без корреляции от структуры каналов
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    if structure_name == "1-канал":
                        continue
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    if not structure_data.empty and 'Изменение H для прибытия (с корреляцией) относительно без корреляции (%)' in structure_data.columns:
                        axes[idx, 2].bar(
                            structure_name.replace("канала", ""),  # Удаляем слово "канал"
                            structure_data['Изменение H для прибытия (с корреляцией) относительно без корреляции (%)'].values[0],
                            label=f"{structure_name}",
                            color=colors[i]
                        )
                axes[idx, 2].set_title(f"Изменение H для прибытия (с корреляцией) для q_nv = {q_nv}", fontsize=12, pad=10)
                axes[idx, 2].set_xlabel("Структура", fontsize=10)
                axes[idx, 2].set_ylabel("Значение в % для H прибытия", fontsize=10)
                axes[idx, 2].legend(fontsize=8)
                axes[idx, 2].grid(True)

                # График 4: Изменение H для отправления (с корреляцией) относительно без корреляции от структуры каналов
                for i, structure_name in enumerate(CHANNEL_STRUCTURES.keys()):
                    if structure_name == "1-канал":
                        continue
                    structure_data = cargo_data[cargo_data['Структура каналов'] == structure_name]
                    if not structure_data.empty and 'Изменение H для отправления (с корреляцией) относительно без корреляции (%)' in structure_data.columns:
                        axes[idx, 3].bar(
                            structure_name.replace("канала", ""),  # Удаляем слово "канал"
                            structure_data['Изменение H для отправления (с корреляцией) относительно без корреляции (%)'].values[0],
                            label=f"{structure_name}",
                            color=colors[i]
                        )
                axes[idx, 3].set_title(f"Изменение H для отправления (с корреляцией) для q_nv = {q_nv}", fontsize=12, pad=10)
                axes[idx, 3].set_xlabel("Структура", fontsize=10)
                axes[idx, 3].set_ylabel("Значение в % для H отправления", fontsize=10)
                axes[idx, 3].legend(fontsize=8)
                axes[idx, 3].grid(True)

            plt.tight_layout(pad=3.0)  # Увеличиваем пространство между графиками
            pdf.savefig(fig)
            plt.close()

    print(f"Графики изменений с корреляцией относительно без корреляции сохранены в файл {pdf_path}")




# Главная функция
def main():
    df = generate_dataset()
    save_dataset(df)

    results_df = calculate_cargo_deviations_combined()

    excel_path = 'cargo_deviations_combined.xlsx'
    results_df.to_excel(excel_path, index=False, sheet_name='Расчеты')
    print(f"Результаты сохранены в файл {excel_path}")

    # Вычисляем изменения и сохраняем их
    changes_df = calculate_and_save_changes(results_df)
   # Сохраняем граафики частот генерации
    df = generate_dataset()
    with PdfPages('frequency_plots.pdf') as pdf_pages:
        plot_frequencies(df, pdf_pages)
    # Строим графики
    plot_sigma_and_H(results_df)
    plot_changes(changes_df)  # Передаем changes_df вместо results_df
    plot_changes_with_corr_vs_without_corr(changes_df)  # Передаем changes_df вместо results_df

if __name__ == "__main__":
    main()




Данные сохранены в файл generated_data.csv
Результаты сохранены в файл cargo_deviations_combined.xlsx
Изменения сохранены в файл cargo_deviations_changes.xlsx
Графики сохранены в файл cargo_analysis.pdf
Графики изменений сохранены в файл changes_analysis.pdf
Графики изменений с корреляцией относительно без корреляции сохранены в файл changes_with_corr_vs_without_corr.pdf


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

# Константы
Q_VALUES = [50.0, 100.0, 150.0, 200.0, 250.0, 300.0, 350.0, 400.0, 450.0, 500.0]
CHANNEL_STRUCTURES = {
    "1-канал": {"channels": 1, "distribution": [1.0]},
    "2-канала (10/90)": {"channels": 2, "distribution": [0.1, 0.9]},
    "2-канала (30/70)": {"channels": 2, "distribution": [0.3, 0.7]},
    "2-канала (50/50)": {"channels": 2, "distribution": [0.5, 0.5]},
    "3-канала (10/20/70)": {"channels": 3, "distribution": [0.1, 0.2, 0.7]},
    "3-канала (20/30/50)": {"channels": 3, "distribution": [0.2, 0.3, 0.5]},
    "3-канала (33.3)": {"channels": 3, "distribution": [1 / 3, 1 / 3, 1 / 3]}
}

# Функция для генерации данных
def generate_dataset():
    data = []
    days_in_year = 365
    for q in Q_VALUES:
        for structure_name, structure in CHANNEL_STRUCTURES.items():
            distribution = structure["distribution"]
            num_channels = structure["channels"]
            avg_daily_flow = q
            channel_flows = [avg_daily_flow * dist for dist in distribution]
            for day in range(1, days_in_year + 1):
                for channel_idx in range(num_channels):
                    intended_mean = channel_flows[channel_idx]
                    value = np.random.normal(loc=intended_mean, scale=intended_mean * 0.05)
                    data.append({
                        "q": q,
                        "Структура каналов": structure_name,
                        "День": day,
                        "Канал": channel_idx + 1,
                        "Значение": value
                    })
    return pd.DataFrame(data)

# Функция для построения графиков частот
def plot_frequencies(df, pdf_pages):
    for structure_name, structure in CHANNEL_STRUCTURES.items():
        num_channels = structure["channels"]
        if num_channels == 1:
            fig, ax = plt.subplots(figsize=(10, 5))
            channel_data = df[(df['Структура каналов'] == structure_name) & (df['Канал'] == 1)]['Значение']
            sns.histplot(channel_data, kde=True, ax=ax)
            ax.set_title(f"Частотный график для структуры каналов: {structure_name}, Канал 1")
            ax.set_xlabel("Значение")
            ax.set_ylabel("Частота")
            plt.tight_layout()
            pdf_pages.savefig(fig)
            plt.close(fig)
        else:
            fig, axes = plt.subplots(1, num_channels, figsize=(15, 5))
            fig.suptitle(f"Частотные графики для структуры каналов: {structure_name}")
            for channel_idx in range(num_channels):
                channel_data = df[(df['Структура каналов'] == structure_name) & (df['Канал'] == channel_idx + 1)]['Значение']
                sns.histplot(channel_data, kde=True, ax=axes[channel_idx])
                axes[channel_idx].set_title(f"Канал {channel_idx + 1}")
                axes[channel_idx].set_xlabel("Значение")
                axes[channel_idx].set_ylabel("Частота")
            plt.tight_layout()
            pdf_pages.savefig(fig)
            plt.close(fig)

# Основная функция
def main():
    df = generate_dataset()
    with PdfPages('frequency_plots.pdf') as pdf_pages:
        plot_frequencies(df, pdf_pages)

if __name__ == "__main__":
    main()

